In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load plotting style
sys.path.append('/home/tobiac/polybox/studium/mscThesis/personal_repo/data_visualization/src')
from mscPlotting import set_plotting_environment, line_properties_generator, SizeParams
set_plotting_environment()

# Load convergence plot tools
from mscPlotting import prepare_convergence_plots, prepare_asymptotic_plots, order_lines, order_exp

# Lineplot Kwargs specific to this Notebook
lineplot_kwargs_dict = {
                   'logx' : True, 
                   'logy' : True, 
                   'linewidth':(0.9),
                   'color' : None,
                   'markers':False,
                   'markersize':7,
                   'markevery':6e-2}

figure_dir = '/home/tobiac/polybox/studium/mscThesis/personal_repo/report/figures/tmp/'

# Templated Hessian

In [ ]:
error_dict_tobia = {"mixed" : pd.read_csv("error_gaussian_mixed.csv"),
            "centered" : pd.read_csv("error_gaussian_centered.csv"),
            "forward" : pd.read_csv("error_gaussian_forward.csv")}
error_dict_tobia["mixed"]

### Hessian Matrix average comparison of different stencils

In [ ]:
fig, ax = plt.subplots()

# Restart generator object for each plot and add current axis
lineplot_kwargs_dict['ax'] = ax
lineplot_kwargs = line_properties_generator(lineplot_kwargs_dict)

first_ycol = 'forward'

cols = next(iter(error_dict_tobia.values())).columns
y_max = 100.0
for key, err_df in error_dict_tobia.items():
    ax.loglog(err_df[cols[0]], err_df[cols[1]], marker=".", markersize=12)
    y_max = max(err_df[cols[1]].max(), y_max)

cols = next(iter(error_dict_tobia.values())).columns
sizes = error_dict_tobia["mixed"][cols[0]]
y0 = error_dict_tobia["forward"][cols[1]][0]
intercepts = [y0, y0]
slopes = [1.0, 2.0]
order_lines(ax, sizes, intercepts, slopes)


labels = [r'mixed', r'centered', r'forward',  r'$1^{\text{st}}$ Order', r'$2^{\text{nd}}$ Order']

prepare_asymptotic_plots(ax, error_dict_tobia["mixed"][cols[0]], labels)

# plt.savefig(figure_dir + 'customHessian_convergence.pdf', dpi=200, bbox_inches="tight")

## Individual Matrix Elements

In [ ]:
def abline(ax, slope, intercept=None):
    """Plot a line from slope and intercept"""
    x_vals = np.array(ax.get_xlim())

    if intercept is None:
        intercept = ax.get_ylim()[1] - slope * x_vals[1]
    y_vals = intercept + slope * x_vals
    ax.plot(x_vals, y_vals, '--')

In [ ]:
def order_lines(sizes, intercepts, slopes, ax):
    for intercept, slope in zip(intercepts, slopes):
        order = np.zeros(len(sizes))
        factor = intercept*(sizes[0]**slope)
        for i, elem in enumerate(sizes):
            order[i] = factor/(elem**slope)

        plt.plot(sizes, order, '--')

In [ ]:
cols = next(iter(error_dict_tobia.values())).columns
fig, ax = plt.subplots()

col_selection = [2, 3, 4]
for col in col_selection:
    error_dict_tobia["mixed"].plot(x=cols[0], y=cols[col], ax=ax, marker=".", markersize=12)

ax.set_xscale("log")
ax.set_yscale("log")

sizes = error_dict_tobia["mixed"][cols[0]]
y0 = error_dict_tobia["mixed"][cols[2]][0]
intercepts = [y0, y0]
slopes = [1.0, 2.0]
order_lines(sizes, intercepts, slopes, ax)

plt.xlabel(f"Gridsize $N$", fontsize=12)
plt.ylabel("Average Relative Error", fontsize=12)
plt.title("Hessian operator on Gaussian in 3D", fontsize=14)
plt.legend([*error_dict_tobia["centered"].keys()[col_selection], "2nd Order", "1st Order"], fontsize=12)
plt.grid()
# plt.savefig('./gaussian_els.pdf', dpi=200, bbox_inches="tight")